# Modkit dmr - Unedit K562 vs unedit T cells day 35 data
## Use my NOT noFilter Reads (epiCG collection .bam, but NO quality condtrol additional reads filtering)

Based on:
https://nanoporetech.github.io/modkit/intro_dmr.html#perform-differential-methylation-scoring

Select kernal: dimelo_v2_modkit_parsing

In [1]:
from datetime import datetime
def current_time():
    """Returns the current date and time as a formatted string."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S") 
    
print("Current Date and Time:", current_time())

Current Date and Time: 2025-11-18 08:33:37


Preparing the input data
The inputs to all modkit dmr commands are two or more bedMethyl files (created by modkit pileup) that have been compressed with bgzip and indexed with tabix. An example of how to generate the input data is shown below:


ref=grch38.fasta
threads=32

norm=normal_sample.bam
norm_pileup=normal_pileup.bed

modkit pileup ${norm} ${norm_pileup} \
  --cpg \
  --ref ${ref} \
  --threads ${threads} \
  --log-filepath log.txt

bgzip -k ${norm_pileup}
tabix -p bed ${norm_pileup}.gz

# pileup and compression can also be done in one step
tumor=tumor_sample.bam
tumor_pileup=tumor_pileup.bed.gz

modkit pileup ${tumor} - \
  --cpg \
  --ref ${ref} \
  --threads ${threads} \
  --log-filepath log.txt | ${bgzip} -c > ${tumor_pileup}

tabix -p bed ${tumor_pileup}

In [2]:
%%bash
echo "hello"

hello


# Use the NEW modkit latest installed version in ipython kernel modkit_new

In [3]:
# ! python3 -m ipykernel install --user --name=modkit_new --display-name "modkit_new Python"
# ! which modkit

In [4]:
import os
os.environ["PATH"] = "/home/michalula/.cargo/bin:" + os.environ["PATH"]
! which modkit
! modkit --version

/home/michalula/.cargo/bin/modkit
modkit 0.5.1


In [5]:
# ! modkit

In [6]:
! modkit --version 

modkit 0.5.1


In [7]:
1+1

2

In [8]:
import os
import pandas as pd

def load_pileup_bed(bed_path):
    # bed_path = existing[0]
    print("Reading bedMethyl file:", bed_path)

    # bedMethyl column names (18 columns as provided)
    colnames = [
        "chrom", "start", "end", "mod_code", "score", "strand",
        "start2", "end2", "color",
        "Nvalid_cov", "percent_modified", "Nmod", "Ncanonical",
        "Nother_mod", "Ndelete", "Nfail", "Ndiff", "Nnocall"
    ]

    # Configure dtypes where reasonable
    dtypes = {
        "chrom": str,
        "start": "Int64",
        "end": "Int64",
        "mod_code": str,
        "score": "Int64",
        "strand": str,
        "start2": "Int64",
        "end2": "Int64",
        "color": str,
        "Nvalid_cov": "Int64",
        "percent_modified": float,
        "Nmod": "Int64",
        "Ncanonical": "Int64",
        "Nother_mod": "Int64",
        "Ndelete": "Int64",
        "Nfail": "Int64",
        "Ndiff": "Int64",
        "Nnocall": "Int64"
    }

    compression = "gzip" if bed_path.endswith(".gz") else None

    # Read file (headerless BED-like table). If file has extra columns, keep them with automatic numeric conversion below.
    df = pd.read_csv(
        bed_path,
        sep="\t",
        header=None,
        comment="#",
        names=colnames,
        dtype=dtypes,
        compression=compression,
        engine="python",
        na_values=[".", "NA", ""],
        keep_default_na=True
    )

    # If file contained more than 18 columns, pandas assigned remaining data to extra columns named like col_18, col_19...
    # Ensure numeric conversion for numeric-like columns
    for c in df.columns:
        if df[c].dtype == object:
            # try safe numeric conversion where appropriate
            try:
                df[c] = pd.to_numeric(df[c], errors="ignore")
            except Exception:
                pass

    print("Loaded DataFrame shape:", df.shape)
    display(df.head())
    return df


In [9]:
import os
from IPython.display import display, HTML
from plotly import express as px
from plotly import graph_objects as go

# ! python3 -m pip install plotly
# ! python3 -m pip install matplotlib
# ! python3 -m pip install nbformat>=4.2.0

def plot_pileup_roi_df(df_roi, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    # ensure numeric types for plotting
    df_roi['pos'] = df_roi['start'].astype(int)
    df_roi['percent_modified'] = df_roi['percent_modified'].astype(float)
    df_roi['Nvalid_cov'] = df_roi['Nvalid_cov'].astype(int)
    df_roi['Nmod'] = df_roi['Nmod'].astype(int)
    df_roi['Ncanonical'] = df_roi['Ncanonical'].astype(int)

    # Scatter: genomic position vs percent modified (point size = coverage)
    fig1 = px.scatter(
        df_roi,
        x='pos',
        y='percent_modified',
        color='strand',
        size='Nvalid_cov',
        hover_data=['Nvalid_cov','Nmod','Ncanonical','Nother_mod','Nnocall'],
        title='Percent modified across ROI (size = Nvalid_cov)',
        height=500
    )
    fig1.update_layout(xaxis_title='Genomic position (start)', yaxis_title='Percent modified')
    fig1.show()
    # fig1.write_html(os.path.join(out_dir, "roi_percent_modified_scatter.html"), include_plotlyjs='cdn')

    # Histogram: coverage distribution
    fig2 = px.histogram(
        df_roi,
        x='Nvalid_cov',
        nbins=40,
        title='Distribution of Nvalid_cov (coverage) in ROI',
        height=400
    )
    fig2.update_layout(xaxis_title='Nvalid_cov', yaxis_title='Count')
    fig2.show()
    # fig2.write_html(os.path.join(out_dir, "roi_nvalidcov_hist.html"), include_plotlyjs='cdn')

    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked)
    topn = 30
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    fig3 = go.Figure()
    fig3.add_trace(go.Bar(name='Nmod', x=df_top['label'], y=df_top['Nmod']))
    fig3.add_trace(go.Bar(name='Ncanonical', x=df_top['label'], y=df_top['Ncanonical']))
    fig3.update_layout(barmode='stack', title=f'Sorted Top {topn} sites by percent_modified (stacked Nmod / Ncanonical)',
                    xaxis_title='position:strand', yaxis_title='reads', height=520)
    fig3.show()
    # fig3.write_html(os.path.join(out_dir, "roi_top_sites_stacked_counts.html"), include_plotlyjs='cdn')

    # Print simple summaries
    print("ROI rows:", df_roi.shape[0])
    print("Percent modified: median={:.2f}, mean={:.2f}".format(df_roi['percent_modified'].median(), df_roi['percent_modified'].mean()))
    print("Coverage (Nvalid_cov): min={}, median={}, max={}".format(df_roi['Nvalid_cov'].min(), df_roi['Nvalid_cov'].median(), df_roi['Nvalid_cov'].max()))

    # Display first rows table for quick inspection
    display(HTML(df_roi.head(20).to_html(index=False)))


    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked)
    topn = df_roi.shape[0]
    # df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_roi.copy() #.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    fig3 = go.Figure()
    fig3.add_trace(go.Bar(name='Nmod', x=df_top['label'], y=df_top['Nmod']))
    fig3.add_trace(go.Bar(name='Ncanonical', x=df_top['label'], y=df_top['Ncanonical']))
    fig3.update_layout(barmode='stack', title=f'All {topn} CpG sites by percent_modified (stacked Nmod / Ncanonical) [ordered=not s]',
                    xaxis_title='position:strand', yaxis_title='reads', height=520)
    fig3.show()
    # fig3.write_html(os.path.join(out_dir, "roi_top_sites_stacked_counts.html"), include_plotlyjs='cdn')

    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked, NOT SORTED)
    topn = df_roi.shape[0]
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    fig3 = go.Figure()
    fig3.add_trace(go.Bar(name='Nmod', x=df_top['label'], y=df_top['Nmod']))
    fig3.add_trace(go.Bar(name='Ncanonical', x=df_top['label'], y=df_top['Ncanonical']))
    fig3.update_layout(barmode='stack', title=f'Sorted Top {topn} sites by percent_modified (stacked Nmod / Ncanonical)',
                    xaxis_title='position:strand', yaxis_title='reads', height=520)
    fig3.show()
    # fig3.write_html(os.path.join(out_dir, "roi_top_sites_stacked_counts.html"), include_plotlyjs='cdn')

    # Print simple summaries
    print("ROI rows:", df_roi.shape[0])
    print("Percent modified: median={:.2f}, mean={:.2f}".format(df_roi['percent_modified'].median(), df_roi['percent_modified'].mean()))
    print("Coverage (Nvalid_cov): min={}, median={}, max={}".format(df_roi['Nvalid_cov'].min(), df_roi['Nvalid_cov'].median(), df_roi['Nvalid_cov'].max()))

    # Display first rows table for quick inspection
    display(HTML(df_roi.head(20).to_html(index=False)))

    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked) percentages
    topn = 30
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    df_top = df_top.assign(Ntotal=df_top['Nmod'] + df_top['Ncanonical'])
    df_top = df_top.assign(Nmod_perc=(df_top['Nmod'] / df_top['Ntotal']) * 100)
    df_top = df_top.assign(Ncanonical_perc=(df_top['Ncanonical'] / df_top['Ntotal']) * 100)
    fig4 = go.Figure()
    fig4.add_trace(go.Bar(name='Nmod %', x=df_top['label'], y=df_top['Nmod_perc']))
    fig4.add_trace(go.Bar(name='Ncanonical %', x=df_top['label'], y=df_top['Ncanonical_perc']))
    fig4.update_layout(barmode='stack', title=f'Top {topn} sites by percent_modified (stacked Nmod % / Ncanonical %)',
                    xaxis_title='position:strand', yaxis_title='percentage', height=520)
    fig4.show()
    # fig4.write_html(os.path.join(out_dir, "roi_top_sites_stacked_percentage.html"), include_plotlyjs='cdn')     


    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked) percentages
    topn = 277
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    df_top = df_top.assign(Ntotal=df_top['Nmod'] + df_top['Ncanonical'])
    df_top = df_top.assign(Nmod_perc=(df_top['Nmod'] / df_top['Ntotal']) * 100)
    df_top = df_top.assign(Ncanonical_perc=(df_top['Ncanonical'] / df_top['Ntotal']) * 100)
    fig4 = go.Figure()
    fig4.add_trace(go.Bar(name='Nmod %', x=df_top['label'], y=df_top['Nmod_perc']))
    fig4.add_trace(go.Bar(name='Ncanonical %', x=df_top['label'], y=df_top['Ncanonical_perc']))
    fig4.update_layout(barmode='stack', title=f'Top {topn} sites by percent_modified (stacked Nmod % / Ncanonical %)',
                    xaxis_title='position:strand', yaxis_title='percentage', height=520)
    fig4.show()
    # fig4.write_html(os.path.join(out_dir, "roi_top_sites_stacked_percentage.html"), include_plotlyjs='cdn')     

    # Bar: Unsorted sites by percent_modified showing Nmod vs Ncanonical (stacked) percentages
    df_top = df_roi.copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    df_top = df_top.assign(Ntotal=df_top['Nmod'] + df_top['Ncanonical'])
    df_top = df_top.assign(Nmod_perc=(df_top['Nmod'] / df_top['Ntotal']) * 100)
    df_top = df_top.assign(Ncanonical_perc=(df_top['Ncanonical'] / df_top['Ntotal']) * 100)
    fig5 = go.Figure()
    fig5.add_trace(go.Bar(name='Nmod %', x=df_top['label'], y=df_top['Nmod_perc']))
    fig5.add_trace(go.Bar(name='Ncanonical %', x=df_top['label'], y=df_top['Ncanonical_perc']))
    fig5.update_layout(barmode='stack', title=f'All sites by percent_modified (stacked Nmod % / Ncanonical %)',
                    xaxis_title='position:strand', yaxis_title='percentage', height=520)
    fig5.show()
    # fig5.write_html(os.path.join(out_dir, "roi_all_sites_stacked_percentage.html"), include_plotlyjs='cdn')    


    return df_top



In [10]:
! ls /home/michalula/data/ref_genomes/t2t_v2_0/

chm13v2.0.fa	  chm13v2.0.fa.fai		   haplotype_vcf
chm13v2.0.fa.amb  chm13v2.0.fa.pac		   up_chm13v2.0.fasta
chm13v2.0.fa.ann  chm13v2.0.fa.sa		   up_chm13v2.0.fasta.fai
chm13v2.0.fa.bwt  convert_to_uppercase_fasta.bash


# K562 Unedited  - postSort Low:

In [13]:
! ls "/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/"

CG_137_padded_reads_K562_unedited_Day2_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.7_T2Tv2_0_chr1:206583354-206589854_2025-11-09_units_combined_numFWD88_numRVS118.npy
CG_137_padded_reads_K562_unedited_Day2_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.995_T2Tv2_0_chr1:206583354-206589854_2025-11-09_units_combined_numFWD88_numRVS118.npy
extracted_reads
filtered_reads_overlap_MORE_than_0.9_K562_unedited_Day2_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam
filtered_reads_overlap_MORE_than_0.9_K562_unedited_Day2_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam.bai
filtered_reads_overlap_MORE_than_0.9_K562_unedited_Day2_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam
filtered_reads_overlap_MORE_than_0.9_K562_unedited_Day2_po

In [47]:
%%bash

ref_genome_fa="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
# ref= "/home/michalula/data/ref_genomes/to_t2t_v1_1/chm13.draft_v1.1.fasta"

threads=32

date_today="20251118"
# data_folder_path="/home/michalula/data/cas9_nanopore/data/20250908_nCATs_T_CRoff_Day_35/5mCG/to_t2t_v2_0/"
# CROFF_day35_bam=${data_folder_path}"sort_align_t2t_v2_0_trim_20250908_Day35_CROFF_Tcells_2Libraries_Minion_R9.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=${data_folder_path}"new_dmr_pileup/"
mkdir ${pileup_data_folder_path}

# mC > 99.5% = 0995 noFilter 
# mC >70% = 07 noFilter 
K562_unedit_noFilter_bam=${data_folder_path}"pre_filtered_ROI_reads_K562_unedited_Day2_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
K562_unedit_noFilter_bed=${pileup_data_folder_path}${date_today}"_pileup""_unedit_K562_noFilter_mC07.bed"
# "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_CROFF_Day28/mergered_outputs/pileup_sort_merge_sort_align_t2t_v1_1_trim_20250721_nCATs_Tcells_CROFF_Day28_minion_run2_day8.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"
# norm=normal_sample.bam
# norm_pileup=normal_pileup.bed

modkit pileup ${K562_unedit_noFilter_bam} ${K562_unedit_noFilter_bed} \
  --cpg \
  --ref ${ref_genome_fa} \
  --threads ${threads} \
  --log-filepath log.txt

bgzip -k ${K562_unedit_noFilter_bed}
tabix -p bed ${K562_unedit_noFilter_bed}.gz

printf '%s\n' "K562_unedit_noFilter_bam: $K562_unedit_noFilter_bam"
printf '%s\n' "K562_unedit_noFilter_bed: $K562_unedit_noFilter_bed"
cat "$K562_unedit_noFilter_bed"


mkdir: cannot create directory ‘/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/’: File exists
> calculated chunk size: 48, interval size 100000, processing 4800000 positions concurrently
> filtering to only CpG motifs
> attempting to sample 10042 reads
> Using filter threshold 0.7128906 for C.
> Done, processed 286 rows. Processed ~240 reads and skipped zero reads.


K562_unedit_noFilter_bam: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/pre_filtered_ROI_reads_K562_unedited_Day2_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam
K562_unedit_noFilter_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251118_pileup_unedit_K562_noFilter_mC07.bed
chr1	206583089	206583090	m	6	+	206583089	206583090	255,0,0	6	0.00	0	6	0	0	0	0	3
chr1	206583090	206583091	m	2	-	206583090	206583091	255,0,0	2	0.00	0	2	0	1	0	0	0
chr1	206583173	206583174	m	99	+	206583173	206583174	255,0,0	99	76.77	76	23	0	1	12	10	12
chr1	206583174	206583175	m	83	-	206583174	206583175	255,0,0	83	89.16	74	9	0	2	2	1	8
chr1	206583387	206583388	m	105	+	206583387	206583388	255,0,0	105	5.71	6	99	0	4	13	9	9
chr1	206583388	206583389	m	52	-	206583388	206583389	255,0,0	52	19.23	10	42	

## Pileup columns explore

In [57]:
date_today="20251118"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=data_folder_path+"new_dmr_pileup/"
# mkdir ${pileup_data_folder_path}

K562_unedit_noFilter_bam="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/pre_filtered_ROI_reads_K562_unedited_Day2_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
K562_unedit_noFilter_bed="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251118_pileup_unedit_K562_noFilter_mC07.bed"

K562_unedit_noFilter_bam, K562_unedit_noFilter_bed

('/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/pre_filtered_ROI_reads_K562_unedited_Day2_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam',
 '/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251118_pileup_unedit_K562_noFilter_mC07.bed')

In [58]:
pileup_K562_unedit_noFilter_pileup_df = load_pileup_bed(K562_unedit_noFilter_bed)
pileup_K562_unedit_noFilter_pileup_df

Reading bedMethyl file: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251118_pileup_unedit_K562_noFilter_mC07.bed
Loaded DataFrame shape: (286, 18)


/tmp/ipykernel_444067/825547292.py:60: FutureWarning:

errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead



,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
0,chr1,206583089,206583090,m,6,+,206583089,206583090,"255,0,0",6,0.00,0,6,0,0,0,0,3
1,chr1,206583090,206583091,m,2,-,206583090,206583091,"255,0,0",2,0.00,0,2,0,1,0,0,0
2,chr1,206583173,206583174,m,99,+,206583173,206583174,"255,0,0",99,76.77,76,23,0,1,12,10,12
3,chr1,206583174,206583175,m,83,-,206583174,206583175,"255,0,0",83,89.16,74,9,0,2,2,1,8
4,chr1,206583387,206583388,m,105,+,206583387,206583388,"255,0,0",105,5.71,6,99,0,4,13,9,9


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
0,chr1,206583089,206583090,m,6,+,206583089,206583090,"255,0,0",6,0.00,0,6,0,0,0,0,3
1,chr1,206583090,206583091,m,2,-,206583090,206583091,"255,0,0",2,0.00,0,2,0,1,0,0,0
2,chr1,206583173,206583174,m,99,+,206583173,206583174,"255,0,0",99,76.77,76,23,0,1,12,10,12
3,chr1,206583174,206583175,m,83,-,206583174,206583175,"255,0,0",83,89.16,74,9,0,2,2,1,8
4,chr1,206583387,206583388,m,105,+,206583387,206583388,"255,0,0",105,5.71,6,99,0,4,13,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,chr1,206589931,206589932,m,10,-,206589931,206589932,"255,0,0",10,90.00,9,1,0,0,1,1,0
282,chr1,206589955,206589956,m,24,+,206589955,206589956,"255,0,0",24,87.50,21,3,0,0,2,2,1
283,chr1,206589956,206589957,m,8,-,206589956,206589957,"255,0,0",8,87.50,7,1,0,2,0,1,0
284,chr1,206590032,206590033,m,22,+,206590032,206590033,"255,0,0",22,90.91,20,2,0,1,1,2,3


# Look at CpGs within out target ROI
T2T v2.0

First CG:
206583388,206583390

Last of selected 137 CGs in the ROI:

206589746,206589748 --CpG_137

=> here have each CG position separate: so have 137*2  = 276

In [59]:
137*2, 277-5

(274, 272)

In [60]:
pileup_K562_unedit_noFilter_pileup_df[pileup_K562_unedit_noFilter_pileup_df['start'] == 206583387]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
4,chr1,206583387,206583388,m,105,+,206583387,206583388,"255,0,0",105,5.71,6,99,0,4,13,9,9


In [61]:
pileup_K562_unedit_noFilter_pileup_df[pileup_K562_unedit_noFilter_pileup_df['start'] == 206583388]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
5,chr1,206583388,206583389,m,52,-,206583388,206583389,"255,0,0",52,19.23,10,42,0,6,2,2,37


In [62]:
pileup_K562_unedit_noFilter_pileup_df[pileup_K562_unedit_noFilter_pileup_df['start'] == 206589746]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
277,chr1,206589746,206589747,m,69,-,206589746,206589747,"255,0,0",69,98.55,68,1,0,12,3,6,9


In [63]:
(279-5) / 2

137.0

In [64]:
pileup_K562_unedit_noFilter_pileup_df_roi = pileup_K562_unedit_noFilter_pileup_df.iloc[4:278, :]  # Display target region rows
print(pileup_K562_unedit_noFilter_pileup_df_roi.shape, pileup_K562_unedit_noFilter_pileup_df_roi.shape[0]/2)
pileup_K562_unedit_noFilter_pileup_df_roi

(274, 18) 137.0


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
4,chr1,206583387,206583388,m,105,+,206583387,206583388,"255,0,0",105,5.71,6,99,0,4,13,9,9
5,chr1,206583388,206583389,m,52,-,206583388,206583389,"255,0,0",52,19.23,10,42,0,6,2,2,37
6,chr1,206583707,206583708,m,103,+,206583707,206583708,"255,0,0",103,11.65,12,91,0,9,3,4,22
7,chr1,206583708,206583709,m,75,-,206583708,206583709,"255,0,0",75,9.33,7,68,0,2,1,10,11
8,chr1,206583766,206583767,m,94,+,206583766,206583767,"255,0,0",94,18.09,17,77,0,4,0,9,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,chr1,206589213,206589214,m,89,-,206589213,206589214,"255,0,0",89,88.76,79,10,0,1,4,0,5
274,chr1,206589436,206589437,m,128,+,206589436,206589437,"255,0,0",128,82.81,106,22,0,1,3,5,4
275,chr1,206589437,206589438,m,79,-,206589437,206589438,"255,0,0",79,84.81,67,12,0,1,3,6,10
276,chr1,206589745,206589746,m,107,+,206589745,206589746,"255,0,0",107,99.07,106,1,0,12,2,5,14


<!>
> Threshold of  0.7597656 for base C is low. Consider increasing the filter-percentile or specifying a higher threshold.
> Done, processed 11762972 rows. Processed ~129977 reads and skipped ~150 reads.

In [56]:
# Plot pileup_Unedit_day35_df_roi summary plots (use existing variables/imports in the notebook)
# Saves interactive HTMLs to pileup_data_folder_path and displays inline.

pileup_K562_unedit_noFilter_pileup_df_roi_stats = plot_pileup_roi_df(df_roi=pileup_K562_unedit_noFilter_pileup_df_roi, out_dir=pileup_data_folder_path)
pileup_K562_unedit_noFilter_pileup_df_roi_stats


/tmp/ipykernel_444067/1063958031.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_444067/1063958031.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_444067/1063958031.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipyke

ROI rows: 274
Percent modified: median=4.57, mean=17.99
Coverage (Nvalid_cov): min=20, median=94.0, max=137


chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
chr1,206583387,206583388,m,105,+,206583387,206583388,"255,0,0",105,5.71,6,99,0,4,13,9,9,206583387
chr1,206583388,206583389,m,52,-,206583388,206583389,"255,0,0",52,19.23,10,42,0,6,2,2,37,206583388
chr1,206583707,206583708,m,103,+,206583707,206583708,"255,0,0",103,11.65,12,91,0,9,3,4,22,206583707
chr1,206583708,206583709,m,75,-,206583708,206583709,"255,0,0",75,9.33,7,68,0,2,1,10,11,206583708
chr1,206583766,206583767,m,94,+,206583766,206583767,"255,0,0",94,18.09,17,77,0,4,0,9,34,206583766
chr1,206583767,206583768,m,80,-,206583767,206583768,"255,0,0",80,35.00,28,52,0,1,7,4,7,206583767
chr1,206584104,206584105,m,118,+,206584104,206584105,"255,0,0",118,72.03,85,33,0,1,0,12,10,206584104
chr1,206584105,206584106,m,84,-,206584105,206584106,"255,0,0",84,77.38,65,19,0,1,3,3,8,206584105
chr1,206584137,206584138,m,134,+,206584137,206584138,"255,0,0",134,74.63,100,34,0,0,5,1,1,206584137
chr1,206584138,206584139,m,67,-,206584138,206584139,"255,0,0",67,82.09,55,12,0,1,28,1,2,206584138


ROI rows: 274
Percent modified: median=4.57, mean=17.99
Coverage (Nvalid_cov): min=20, median=94.0, max=137


chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
chr1,206583387,206583388,m,105,+,206583387,206583388,"255,0,0",105,5.71,6,99,0,4,13,9,9,206583387
chr1,206583388,206583389,m,52,-,206583388,206583389,"255,0,0",52,19.23,10,42,0,6,2,2,37,206583388
chr1,206583707,206583708,m,103,+,206583707,206583708,"255,0,0",103,11.65,12,91,0,9,3,4,22,206583707
chr1,206583708,206583709,m,75,-,206583708,206583709,"255,0,0",75,9.33,7,68,0,2,1,10,11,206583708
chr1,206583766,206583767,m,94,+,206583766,206583767,"255,0,0",94,18.09,17,77,0,4,0,9,34,206583766
chr1,206583767,206583768,m,80,-,206583767,206583768,"255,0,0",80,35.00,28,52,0,1,7,4,7,206583767
chr1,206584104,206584105,m,118,+,206584104,206584105,"255,0,0",118,72.03,85,33,0,1,0,12,10,206584104
chr1,206584105,206584106,m,84,-,206584105,206584106,"255,0,0",84,77.38,65,19,0,1,3,3,8,206584105
chr1,206584137,206584138,m,134,+,206584137,206584138,"255,0,0",134,74.63,100,34,0,0,5,1,1,206584137
chr1,206584138,206584139,m,67,-,206584138,206584139,"255,0,0",67,82.09,55,12,0,1,28,1,2,206584138


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,...,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos,label,Ntotal,Nmod_perc,Ncanonical_perc
4,chr1,206583387,206583388,m,105,+,206583387,206583388,"255,0,0",105,...,0,4,13,9,9,206583387,206583387:+,105,5.714286,94.285714
5,chr1,206583388,206583389,m,52,-,206583388,206583389,"255,0,0",52,...,0,6,2,2,37,206583388,206583388:-,52,19.230769,80.769231
6,chr1,206583707,206583708,m,103,+,206583707,206583708,"255,0,0",103,...,0,9,3,4,22,206583707,206583707:+,103,11.650485,88.349515
7,chr1,206583708,206583709,m,75,-,206583708,206583709,"255,0,0",75,...,0,2,1,10,11,206583708,206583708:-,75,9.333333,90.666667
8,chr1,206583766,206583767,m,94,+,206583766,206583767,"255,0,0",94,...,0,4,0,9,34,206583766,206583766:+,94,18.085106,81.914894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,chr1,206589213,206589214,m,89,-,206589213,206589214,"255,0,0",89,...,0,1,4,0,5,206589213,206589213:-,89,88.764045,11.235955
274,chr1,206589436,206589437,m,128,+,206589436,206589437,"255,0,0",128,...,0,1,3,5,4,206589436,206589436:+,128,82.812500,17.187500
275,chr1,206589437,206589438,m,79,-,206589437,206589438,"255,0,0",79,...,0,1,3,6,10,206589437,206589437:-,79,84.810127,15.189873
276,chr1,206589745,206589746,m,107,+,206589745,206589746,"255,0,0",107,...,0,12,2,5,14,206589745,206589745:+,107,99.065421,0.934579


# Unedited T cells Day 35

In [24]:
! ls "/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output"

CG_137_padded_reads_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.7_T2Tv2_0_chr1:206583354-206589854_2025-11-09_units_combined_numFWD87_numRVS115.npy
CG_137_padded_reads_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.995_T2Tv2_0_chr1:206583354-206589854_2025-11-09_units_combined_numFWD87_numRVS114.npy
extracted_reads
filtered_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam
filtered_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam.bai
filtered_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam
filtered_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_thre

In [26]:
! ls "/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/"

CG_137_padded_reads_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.7_T2Tv2_0_chr1:206583354-206589854_2025-11-09_units_combined_numFWD87_numRVS115.npy
CG_137_padded_reads_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_mCthresh0.995_T2Tv2_0_chr1:206583354-206589854_2025-11-09_units_combined_numFWD87_numRVS114.npy
extracted_reads
filtered_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam
filtered_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam.bai
filtered_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam
filtered_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_thre

In [27]:
%%bash

date_today="20251118"
data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/"
pileup_data_folder_path=${data_folder_path}"new_dmr_pileup/"
mkdir ${pileup_data_folder_path}

# mC > 99.5% = 0995 noFilter 
# noFilter_Unedit_day35_bam=${data_folder_path}"noFilter_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# mC >70% = 07 noFilter   
noFilter_Unedit_day35_bam=${data_folder_path}"pre_filtered_ROI_reads_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
pileup_Unedit_day35_bed=${pileup_data_folder_path}${date_today}"_unedit_T_noFilter_mC07""_pileup_NT_Day35_Tcells.bed"
cat "$pileup_Unedit_day35_bed"

ref_genome_fa="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
# ref="/home/michalula/data/ref_genomes/to_t2t_v1_1/chm13.draft_v1.1.fasta"

threads=32

modkit pileup ${noFilter_Unedit_day35_bam} ${pileup_Unedit_day35_bed} \
  --cpg \
  --ref ${ref_genome_fa} \
  --threads ${threads} \
  --log-filepath log.txt

bgzip -k ${pileup_Unedit_day35_bed}
tabix -p bed ${pileup_Unedit_day35_bed}.gz

printf '%s\n' "noFilter_Unedit_day35_bam: $noFilter_Unedit_day35_bam"
printf '%s\n' "pileup_Unedit_day35_bed: $pileup_Unedit_day35_bed"
cat "$pileup_Unedit_day35_bed"


mkdir: cannot create directory ‘/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/’: File exists
cat: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed: No such file or directory
> calculated chunk size: 48, interval size 100000, processing 4800000 positions concurrently
> filtering to only CpG motifs
> attempting to sample 10042 reads
> Using filter threshold 0.8496094 for C.
> Done, processed 285 rows. Processed ~215 reads and skipped zero reads.


noFilter_Unedit_day35_bam: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/pre_filtered_ROI_reads_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam
pileup_Unedit_day35_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed
chr1	206583090	206583091	m	1	-	206583090	206583091	255,0,0	1	100.00	1	0	0	0	0	0	0
chr1	206583173	206583174	m	80	+	206583173	206583174	255,0,0	80	85.00	68	12	0	3	18	8	4
chr1	206583174	206583175	m	90	-	206583174	206583175	255,0,0	90	95.56	86	4	0	1	3	2	2
chr1	206583387	206583388	m	85	+	206583387	206583388	255,0,0	85	72.94	62	23	0	1	15	10	4
chr1	206583388	206583389	m	75	-	206583388	206583389	255,0,0	75	86.67	65	10	0	3	14	1	5
chr1	206583707	206583708	m	104	+	206583707	2

In [29]:
date_today="20251118"
# data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/"
# pileup_data_folder_path=data_folder_path+"new_dmr_pileup/"
# # mkdir ${pileup_data_folder_path}

# # noFilter_Unedit_day35_bam=${data_folder_path}"noFilter_reads_overlap_MORE_than_0.9_Tcells_NT_Day35_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# pileup_Unedit_day35_bed=pileup_data_folder_path+date_today+"_unedit_T_noFilter_mC07""_pileup_NT_Day35_Tcells.bed"
# print("pileup_Unedit_day35_bed:", pileup_Unedit_day35_bed)
pileup_Unedit_day35_bed="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed"

pileup_Unedit_day35_df = load_pileup_bed(pileup_Unedit_day35_bed)
pileup_Unedit_day35_df

Reading bedMethyl file: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed
Loaded DataFrame shape: (285, 18)


/tmp/ipykernel_444067/825547292.py:60: FutureWarning:

errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead



,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
0,chr1,206583090,206583091,m,1,-,206583090,206583091,"255,0,0",1,100.00,1,0,0,0,0,0,0
1,chr1,206583173,206583174,m,80,+,206583173,206583174,"255,0,0",80,85.00,68,12,0,3,18,8,4
2,chr1,206583174,206583175,m,90,-,206583174,206583175,"255,0,0",90,95.56,86,4,0,1,3,2,2
3,chr1,206583387,206583388,m,85,+,206583387,206583388,"255,0,0",85,72.94,62,23,0,1,15,10,4
4,chr1,206583388,206583389,m,75,-,206583388,206583389,"255,0,0",75,86.67,65,10,0,3,14,1,5


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
0,chr1,206583090,206583091,m,1,-,206583090,206583091,"255,0,0",1,100.00,1,0,0,0,0,0,0
1,chr1,206583173,206583174,m,80,+,206583173,206583174,"255,0,0",80,85.00,68,12,0,3,18,8,4
2,chr1,206583174,206583175,m,90,-,206583174,206583175,"255,0,0",90,95.56,86,4,0,1,3,2,2
3,chr1,206583387,206583388,m,85,+,206583387,206583388,"255,0,0",85,72.94,62,23,0,1,15,10,4
4,chr1,206583388,206583389,m,75,-,206583388,206583389,"255,0,0",75,86.67,65,10,0,3,14,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,chr1,206589931,206589932,m,6,-,206589931,206589932,"255,0,0",6,100.00,6,0,0,0,0,0,0
281,chr1,206589955,206589956,m,6,+,206589955,206589956,"255,0,0",6,100.00,6,0,0,0,2,0,0
282,chr1,206589956,206589957,m,3,-,206589956,206589957,"255,0,0",3,100.00,3,0,0,0,0,0,2
283,chr1,206590032,206590033,m,6,+,206590032,206590033,"255,0,0",6,83.33,5,1,0,0,0,2,0


In [30]:
pileup_Unedit_day35_df[pileup_Unedit_day35_df['start'] == 206583388-1]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
3,chr1,206583387,206583388,m,85,+,206583387,206583388,"255,0,0",85,72.94,62,23,0,1,15,10,4


In [31]:
pileup_Unedit_day35_df[pileup_Unedit_day35_df['start'] == 206583388]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
4,chr1,206583388,206583389,m,75,-,206583388,206583389,"255,0,0",75,86.67,65,10,0,3,14,1,5


In [32]:
pileup_Unedit_day35_df[pileup_Unedit_day35_df['start'] == 206589746-1]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
275,chr1,206589745,206589746,m,101,+,206589745,206589746,"255,0,0",101,99.01,100,1,0,7,0,0,7


In [33]:
pileup_Unedit_day35_df[pileup_Unedit_day35_df['start'] == 206589746]

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
276,chr1,206589746,206589747,m,74,-,206589746,206589747,"255,0,0",74,100.0,74,0,0,9,7,4,6


In [34]:
pileup_Unedit_day35_df_roi = pileup_Unedit_day35_df.iloc[3:277, :]  # Display target region rows
print(pileup_Unedit_day35_df_roi.shape,pileup_Unedit_day35_df_roi.shape[0]/2)
pileup_Unedit_day35_df_roi

(274, 18) 137.0


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
3,chr1,206583387,206583388,m,85,+,206583387,206583388,"255,0,0",85,72.94,62,23,0,1,15,10,4
4,chr1,206583388,206583389,m,75,-,206583388,206583389,"255,0,0",75,86.67,65,10,0,3,14,1,5
5,chr1,206583707,206583708,m,104,+,206583707,206583708,"255,0,0",104,96.15,100,4,0,3,3,0,5
6,chr1,206583708,206583709,m,80,-,206583708,206583709,"255,0,0",80,96.25,77,3,0,1,9,6,3
7,chr1,206583766,206583767,m,97,+,206583766,206583767,"255,0,0",97,92.78,90,7,0,3,1,12,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,chr1,206589213,206589214,m,87,-,206589213,206589214,"255,0,0",87,78.16,68,19,0,2,8,0,3
273,chr1,206589436,206589437,m,110,+,206589436,206589437,"255,0,0",110,95.45,105,5,0,0,4,1,0
274,chr1,206589437,206589438,m,92,-,206589437,206589438,"255,0,0",92,95.65,88,4,0,0,4,3,1
275,chr1,206589745,206589746,m,101,+,206589745,206589746,"255,0,0",101,99.01,100,1,0,7,0,0,7


In [35]:
pileup_Unedit_day35_df_roi

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
3,chr1,206583387,206583388,m,85,+,206583387,206583388,"255,0,0",85,72.94,62,23,0,1,15,10,4
4,chr1,206583388,206583389,m,75,-,206583388,206583389,"255,0,0",75,86.67,65,10,0,3,14,1,5
5,chr1,206583707,206583708,m,104,+,206583707,206583708,"255,0,0",104,96.15,100,4,0,3,3,0,5
6,chr1,206583708,206583709,m,80,-,206583708,206583709,"255,0,0",80,96.25,77,3,0,1,9,6,3
7,chr1,206583766,206583767,m,97,+,206583766,206583767,"255,0,0",97,92.78,90,7,0,3,1,12,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,chr1,206589213,206589214,m,87,-,206589213,206589214,"255,0,0",87,78.16,68,19,0,2,8,0,3
273,chr1,206589436,206589437,m,110,+,206589436,206589437,"255,0,0",110,95.45,105,5,0,0,4,1,0
274,chr1,206589437,206589438,m,92,-,206589437,206589438,"255,0,0",92,95.65,88,4,0,0,4,3,1
275,chr1,206589745,206589746,m,101,+,206589745,206589746,"255,0,0",101,99.01,100,1,0,7,0,0,7


In [36]:
pileup_Unedit_day35_df_roi

,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall
3,chr1,206583387,206583388,m,85,+,206583387,206583388,"255,0,0",85,72.94,62,23,0,1,15,10,4
4,chr1,206583388,206583389,m,75,-,206583388,206583389,"255,0,0",75,86.67,65,10,0,3,14,1,5
5,chr1,206583707,206583708,m,104,+,206583707,206583708,"255,0,0",104,96.15,100,4,0,3,3,0,5
6,chr1,206583708,206583709,m,80,-,206583708,206583709,"255,0,0",80,96.25,77,3,0,1,9,6,3
7,chr1,206583766,206583767,m,97,+,206583766,206583767,"255,0,0",97,92.78,90,7,0,3,1,12,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,chr1,206589213,206589214,m,87,-,206589213,206589214,"255,0,0",87,78.16,68,19,0,2,8,0,3
273,chr1,206589436,206589437,m,110,+,206589436,206589437,"255,0,0",110,95.45,105,5,0,0,4,1,0
274,chr1,206589437,206589438,m,92,-,206589437,206589438,"255,0,0",92,95.65,88,4,0,0,4,3,1
275,chr1,206589745,206589746,m,101,+,206589745,206589746,"255,0,0",101,99.01,100,1,0,7,0,0,7


In [37]:
pileup_data_folder_path

'/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/'

In [38]:

# data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/croff/replica_1/analyze_single_reads/dimelo_v2_output/"
# pileup_data_folder_path=data_folder_path+"new_dmr_pileup/"
# # mkdir ${pileup_data_folder_path}

# # Plot pileup_Unedit_day35_df_roi summary plots (use existing variables/imports in the notebook)
# # Saves interactive HTMLs to pileup_data_folder_path and displays inline.
# out_dir = pileup_data_folder_path  # existing variable in the notebook

# df_roi = pileup_Unedit_day35_df_roi.copy()

df_roi_stats = plot_pileup_roi_df(df_roi=pileup_Unedit_day35_df_roi, out_dir=pileup_data_folder_path)
df_roi_stats


/tmp/ipykernel_444067/1063958031.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_444067/1063958031.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_444067/1063958031.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipyke

ROI rows: 274
Percent modified: median=2.08, mean=22.27
Coverage (Nvalid_cov): min=14, median=91.0, max=113


chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
chr1,206583387,206583388,m,85,+,206583387,206583388,"255,0,0",85,72.94,62,23,0,1,15,10,4,206583387
chr1,206583388,206583389,m,75,-,206583388,206583389,"255,0,0",75,86.67,65,10,0,3,14,1,5,206583388
chr1,206583707,206583708,m,104,+,206583707,206583708,"255,0,0",104,96.15,100,4,0,3,3,0,5,206583707
chr1,206583708,206583709,m,80,-,206583708,206583709,"255,0,0",80,96.25,77,3,0,1,9,6,3,206583708
chr1,206583766,206583767,m,97,+,206583766,206583767,"255,0,0",97,92.78,90,7,0,3,1,12,2,206583766
chr1,206583767,206583768,m,78,-,206583767,206583768,"255,0,0",78,93.59,73,5,0,3,12,2,4,206583767
chr1,206584104,206584105,m,100,+,206584104,206584105,"255,0,0",100,96.00,96,4,0,1,1,12,1,206584104
chr1,206584105,206584106,m,92,-,206584105,206584106,"255,0,0",92,92.39,85,7,0,1,2,1,3,206584105
chr1,206584137,206584138,m,109,+,206584137,206584138,"255,0,0",109,100.00,109,0,0,1,4,0,1,206584137
chr1,206584138,206584139,m,55,-,206584138,206584139,"255,0,0",55,90.91,50,5,0,1,42,0,1,206584138


ROI rows: 274
Percent modified: median=2.08, mean=22.27
Coverage (Nvalid_cov): min=14, median=91.0, max=113


chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,percent_modified,Nmod,Ncanonical,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos
chr1,206583387,206583388,m,85,+,206583387,206583388,"255,0,0",85,72.94,62,23,0,1,15,10,4,206583387
chr1,206583388,206583389,m,75,-,206583388,206583389,"255,0,0",75,86.67,65,10,0,3,14,1,5,206583388
chr1,206583707,206583708,m,104,+,206583707,206583708,"255,0,0",104,96.15,100,4,0,3,3,0,5,206583707
chr1,206583708,206583709,m,80,-,206583708,206583709,"255,0,0",80,96.25,77,3,0,1,9,6,3,206583708
chr1,206583766,206583767,m,97,+,206583766,206583767,"255,0,0",97,92.78,90,7,0,3,1,12,2,206583766
chr1,206583767,206583768,m,78,-,206583767,206583768,"255,0,0",78,93.59,73,5,0,3,12,2,4,206583767
chr1,206584104,206584105,m,100,+,206584104,206584105,"255,0,0",100,96.00,96,4,0,1,1,12,1,206584104
chr1,206584105,206584106,m,92,-,206584105,206584106,"255,0,0",92,92.39,85,7,0,1,2,1,3,206584105
chr1,206584137,206584138,m,109,+,206584137,206584138,"255,0,0",109,100.00,109,0,0,1,4,0,1,206584137
chr1,206584138,206584139,m,55,-,206584138,206584139,"255,0,0",55,90.91,50,5,0,1,42,0,1,206584138


,chrom,start,end,mod_code,score,strand,start2,end2,color,Nvalid_cov,...,Nother_mod,Ndelete,Nfail,Ndiff,Nnocall,pos,label,Ntotal,Nmod_perc,Ncanonical_perc
3,chr1,206583387,206583388,m,85,+,206583387,206583388,"255,0,0",85,...,0,1,15,10,4,206583387,206583387:+,85,72.941176,27.058824
4,chr1,206583388,206583389,m,75,-,206583388,206583389,"255,0,0",75,...,0,3,14,1,5,206583388,206583388:-,75,86.666667,13.333333
5,chr1,206583707,206583708,m,104,+,206583707,206583708,"255,0,0",104,...,0,3,3,0,5,206583707,206583707:+,104,96.153846,3.846154
6,chr1,206583708,206583709,m,80,-,206583708,206583709,"255,0,0",80,...,0,1,9,6,3,206583708,206583708:-,80,96.250000,3.750000
7,chr1,206583766,206583767,m,97,+,206583766,206583767,"255,0,0",97,...,0,3,1,12,2,206583766,206583766:+,97,92.783505,7.216495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,chr1,206589213,206589214,m,87,-,206589213,206589214,"255,0,0",87,...,0,2,8,0,3,206589213,206589213:-,87,78.160920,21.839080
273,chr1,206589436,206589437,m,110,+,206589436,206589437,"255,0,0",110,...,0,0,4,1,0,206589436,206589436:+,110,95.454545,4.545455
274,chr1,206589437,206589438,m,92,-,206589437,206589438,"255,0,0",92,...,0,0,4,3,1,206589437,206589437:-,92,95.652174,4.347826
275,chr1,206589745,206589746,m,101,+,206589745,206589746,"255,0,0",101,...,0,7,0,0,7,206589745,206589745:+,101,99.009901,0.990099


# dmr modkit Unedited K562 vs T cells (NT)

3. Detecting differential modification at single base positions
The modkit dmr pair command has the ability to score individual bases (e.g. differentially methylated CpGs). To run single-base analysis on one or more paired samples, simply omit the --regions (-r) option when running modkit dmr pair. When performing single-base analysis the likelihood ratio score and a MAP-based p-value are available. For details on the likelihood ratio score and the MAP-based p-value, see the scoring details section. For example the above command becomes:

dmr_result=single_base_haplotype_dmr.bed

modkit dmr pair \
  -a ${hp1_pileup}.gz \
  -b ${hp2_pileup}.gz \
  -o ${dmr_result} \
  --ref ${ref} \
  --base C \
  --threads ${threads} \
  --log-filepath dmr.log

In [39]:
# ! ls "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG"

In [40]:
# ! ls /home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup@v3.3.5mCG.bam.gz

In [41]:
# /home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam.gz

In [42]:
# %%bash
# ls -ld /home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs
# ls -l /home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam.gz

In [ ]:
%%bash

pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/"
# "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG/to_t2t_v2_0/pileup/pileup_Unedit_Day28_Tcells_20250721.bam"
pileup_Unedit_day35_bed="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed"

echo "pileup_Unedit_day35_bed: ${pileup_Unedit_day35_bed}.gz"

ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_Unedit_day35_bed}.gz"
# ls -l "${pileup_Unedit_day28_bam}.gz"

chmod u+rwx "${pileup_data_folder_path}"
chmod u+rwx "${pileup_Unedit_day35_bed}.gz"

# ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_Unedit_day35_bed}.gz"


pileup_Unedit_day35_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed.gz
20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed
20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed.gz
20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed.gz.tbi
drwxrwxr-x 2 michalula michalula 4096 Nov 18 08:46 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/
-rw-rw-r-- 1 michalula michalula 5672 Nov 18 08:46 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed.gz
drwxrwxr-x 2 michalula michalula 4096 Nov 18 08:46 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_

In [67]:
%%bash

# pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/pileup/"
# pileup_Unedit_day35_bam=${pileup_data_folder_path}"20251109_noFilter_pileup_NT_Day35_Tcells.bam"
# # "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG/to_t2t_v2_0/pileup/pileup_Unedit_Day28_Tcells_20250721.bam"
pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/"
# K562_unedit_noFilter_bed=${pileup_data_folder_path}"20251109_noFilter_mC07_pileup_CROFF_Day35_Tcells.bed"

# K562_unedit_noFilter_bam="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/pre_filtered_ROI_reads_K562_unedited_Day2_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
K562_unedit_noFilter_bed="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251118_pileup_unedit_K562_noFilter_mC07.bed"

echo "K562_unedit_noFilter_bed: ${K562_unedit_noFilter_bed}.gz"

ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${K562_unedit_noFilter_bed}.gz"
# ls -l "${pileup_Unedit_day28_bam}.gz"

chmod u+rwx "${pileup_data_folder_path}"
chmod u+rwx "${K562_unedit_noFilter_beded}.gz"


# ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${K562_unedit_noFilter_bed}.gz"


K562_unedit_noFilter_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251118_pileup_unedit_K562_noFilter_mC07.bed.gz
20251118_pileup_unedit_K562_noFilter_mC07.bed
20251118_pileup_unedit_K562_noFilter_mC07.bed.gz
20251118_pileup_unedit_K562_noFilter_mC07.bed.gz.tbi
drwxrwxr-x 2 michalula michalula 4096 Nov 18 09:00 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/
-rw-rw-r-- 1 michalula michalula 6159 Nov 18 09:00 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251118_pileup_unedit_K562_noFilter_mC07.bed.gz


chmod: cannot access '.gz': No such file or directory


drwxrwxr-x 2 michalula michalula 4096 Nov 18 09:00 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/
-rw-rw-r-- 1 michalula michalula 6159 Nov 18 09:00 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251118_pileup_unedit_K562_noFilter_mC07.bed.gz


In [74]:
%%bash
 
# 3. Detecting differential modification at single base positions
# The modkit dmr pair command has the ability to score individual bases (e.g. differentially methylated CpGs). To run single-base analysis on one or more paired samples, simply omit the --regions (-r) option when running modkit dmr pair. When performing single-base analysis the likelihood ratio score and a MAP-based p-value are available. For details on the likelihood ratio score and the MAP-based p-value, see the scoring details section. For example the above command becomes:
date_today="20251118"

experiment_codition="unedit_K562_v_T_day35_noFilter_mC07"
dmr_output_path="/home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/"
dmr_result=${dmr_output_path}${date_today}"_single_base_"${experiment_codition}".bed"

# pileup_Unedit_day28_bam="/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"
# removed the @ from the file name
# pileup_Unedit_day28_bam="/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam"
# pileup_Unedit_day28_bam="/home/michalula/code/epiCausality/epiCode/differential_methyl/pileups/t2t_v1_1/pileup_Unedit_Day28_Tcells_20250721.bam"

# pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/"
pileup_Unedit_day35_bed="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed"
# "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG/to_t2t_v2_0/pileup/pileup_Unedit_Day28_Tcells_20250721.bam"

echo "pileup_Unedit_day35_bed: ${pileup_Unedit_day35_bed}.gz"
ls -l "${pileup_Unedit_day35_bed}.gz"
# "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam"
# pileup_Unedit_day28_bam_gz="/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/pileup_sorted_align_t2t_v1_1_trim_20250721_nCATs_Tcells_UNEDITED_Day28_minion_merged.dna_r9.4.1_e8_sup.5mCG.bam.gz"
  
# pileup_CROFF_day28_bam="/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_CROFF_Day28/mergered_outputs/pileup_sort_merge_sort_align_t2t_v1_1_trim_20250721_nCATs_Tcells_CROFF_Day28_minion_run2_day8.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"
# pileup_CROFF_day28_bam="/home/michalula/code/epiCausality/epiCode/differential_methyl/pileups/t2t_v1_1/pileup_CROFF_Day28_Tcells_20250721.bam"
# pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/"
# K562_unedit_noFilter_bed=${pileup_data_folder_path}"20251109_noFilter_mC07_pileup_CROFF_Day35_Tcells.bed"
K562_unedit_noFilter_bed="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251118_pileup_unedit_K562_noFilter_mC07.bed"

echo "K562_unedit_noFilter_bed: ${K562_unedit_noFilter_bed}.gz"
ls -l "${K562_unedit_noFilter_bed}.gz"

ref_genome_fa="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
# ref="/home/michalula/data/ref_genomes/to_t2t_v1_1/chm13.draft_v1.1.fasta"
threads=32
 
cd ${dmr_output_path}
# '/home/michalula/code/epiCausality/epiCode/differential_methyl'

modkit dmr pair \
  -a ${K562_unedit_noFilter_bed}.gz \
  -b ${pileup_Unedit_day35_bed}.gz \
  -o ${dmr_result} \
  --ref ${ref_genome_fa} \
  --base C \
  --threads ${threads} \
  --log-filepath dmr.log


echo "dmr_result: $dmr_result"
ls -lah $dmr_result

pileup_Unedit_day35_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed.gz
-rwxrw-r-- 1 michalula michalula 5672 Nov 18 08:46 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/unedited/merged_2libraries/dimelo_v2_output/new_dmr_pileup/20251118_unedit_T_noFilter_mC07_pileup_NT_Day35_Tcells.bed.gz
K562_unedit_noFilter_bed: /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251118_pileup_unedit_K562_noFilter_mC07.bed.gz
-rw-rw-r-- 1 michalula michalula 6159 Nov 18 09:00 /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/K562/unedited/analyze_single_reads/dimelo_v2_output/new_dmr_pileup/20251118_pileup_unedit_K562_noFilter_mC07.bed.gz


> reading reference FASTA at "/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
> 1 common sequence(s) between FASTA and both samples
> running single-site analysis
> using default prior, Beta(α: 0.55, β: 0.55)
> estimating max coverages from data
> sampled 286 a records and 285 b records, calculating max coverages for 95th percentile
> calculated max coverage for a: 130 and b: 108
> calculated max coverage 130 is greater than maximum allowed (100), setting to 100
> calculated max coverage 108 is greater than maximum allowed (100), setting to 100
> errors:
+--------------------------+-------+
| error                    | count |
+--------------------------+-------+
| missing-in-one-condition | 1     |
+--------------------------+-------+

> finished, processed 285 sites successfully, 1 failed


dmr_result: /home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/20251118_single_base_unedit_K562_v_T_day35_noFilter_mC07.bed
-rw-rw-r-- 1 michalula michalula 51K Nov 18 09:10 /home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/20251118_single_base_unedit_K562_v_T_day35_noFilter_mC07.bed


In [75]:
%%bash

date_today="20251118"
experiment_codition="unedit_K562_v_T_day35_noFilter_mC07"
dmr_output_path="/home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/"
# "/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_noFilter/new_dmr_output/"
dmr_result=${dmr_output_path}"_"${date_today}"single_base_"${experiment_codition}".bed"

echo "dmr_result: $dmr_result"
# ls -lah $dmr_result
ls -lah $dmr_output_path
# cat $dmr_result

dmr_result: /home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/_20251118single_base_unedit_K562_v_T_day35_noFilter_mC07.bed
total 68K
drwxrwxr-x 2 michalula michalula 4.0K Nov 18 09:10 .
drwxrwxr-x 3 michalula michalula 4.0K Nov 18 09:08 ..
-rw-rw-r-- 1 michalula michalula  51K Nov 18 09:10 20251118_single_base_unedit_K562_v_T_day35_noFilter_mC07.bed
-rw-rw-r-- 1 michalula michalula 4.8K Nov 18 09:10 dmr.log


In [77]:
pwd

'/home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered'

## modkit dmr explore output

The full table when performing single-site analysis with equal numbers of samples in groups, when running modkit dmr pair, will have the following schema:

column	name	description	type
1	chrom	name of reference sequence from bedMethyl input samples	str
2	start position	0-based start position, from --regions argument	int
3	end position	0-based exclusive end position, from --regions argument	int
4	name	name column from --regions BED, or chr:start-stop if absent, "." for single sites	str
5	score	difference score, more positive values have increased difference	float
6	strand	strand for the region or single-base position	str
7	samplea counts	counts of each base modification in the region, comma-separated, for sample A	str
8	samplea total	total number of base modification calls in the region, including unmodified, for sample A	int
9	sampleb counts	counts of each base modification in the region, comma-separated, for sample B	str
10	sampleb total	total number of base modification calls in the region, including unmodified, for sample B	int
11	samplea percents	percent of calls for each base modification in the region, comma-separated, for sample A	str
12	sampleb percents	percent of calls for each base modification in the region, comma-separated, for sample B	str
13	samplea fraction modified	fraction modification (of any kind) in sample A	float
14	sampleb fraction modified	fraction modification (of any kind) in sample B	float
15	MAP-based p-value	ratio of the posterior probability of observing the effect size over zero effect size	float
16	effect size	percent modified in sample A (col 12) minus percent modified in sample B (col 13)	float
<!-- 17	balanced MAP-based p-value	MAP-based p-value when all replicates are balanced	float
18	balanced effect size	effect size when all replicates are balanced	float -->


17	cohen_h	Cohen's h statistic (useful with regions and high-depth runs)	float
18	cohen_h_low	95% confidence interval lower bound	float
19	cohen_h_high	95% confidence interval upper bound	float

<!-- Differential methylation output format
The output from modkit dmr pair (and for each pairwise comparison with modkit dmr multi) is (roughly) a BED file with the following schema: -->
<!-- 
column	name	description	type
        1	chrom	name of reference sequence from bedMethyl input samples	str
        2	start position	0-based start position, from --regions argument	int
        3	end position	0-based exclusive end position, from --regions argument	int
        4	name	name column from --regions BED, or chr:start-stop if absent, "." for single sites	str
        5	score	difference score, more positive values have increased difference	float
        6	strand	strand for the region or single-base position	str
        7	samplea counts	counts of each base modification in the region, comma-separated, for sample A	str
        8	samplea total	total number of base modification calls in the region, including unmodified, for sample A	int
        9	sampleb counts	counts of each base modification in the region, comma-separated, for sample B	str
        10	sampleb total	total number of base modification calls in the region, including unmodified, for sample B	int
        11	samplea percents	percent of calls for each base modification in the region, comma-separated, for sample A	str
        12	sampleb percents	percent of calls for each base modification in the region, comma-separated, for sample B	str
        13	samplea fraction modified	fraction modification (of any kind) in sample A	float
        14	sampleb fraction modified	fraction modification (of any kind) in sample B	float
        15	MAP-based p-value	ratio of the posterior probability of observing the effect size over zero effect size	float
        16	effect size	percent modified in sample A (col 12) minus percent modified in sample B (col 13)	float
        17	balanced MAP-based p-value	MAP-based p-value when all replicates are balanced	float
        18	balanced effect size	effect size when all replicates are balanced	float
        19	pct_a_samples	percent of 'a' samples used in statistical test	float
        20	pct_b_samples	percent of 'b' samples used in statistical test	float
        21	per-replicate p-values	MAP-based p-values for matched replicate pairs	float
        22	per-replicate effect sizes	effect sizes matched replicate pairs	float
        23	cohen_h	Cohen's h statistic (useful with regions and high-depth runs)	float
        24	cohen_h_low	95% confidence interval lower bound	float
        25	cohen_h_high	95% confidence interval upper bound	float
        Columns 16-19 are only produced when multiple samples are provided, columns 20 and 21 are only produced when there is an equal number of 'a' and 'b' samples. When using multiple samples, it is possible that not every sample will have a modification fraction at a position. When this happens, the statistical test is still performed and the values of pct_a_samples and pct_b_samples reflect the percent of samples from each condition used in the test. 


     (15)	cohen_h	Cohen's h statistic (useful with regions and high-depth runs)	float
    (16)	cohen_h_low	95% confidence interval lower bound	float
    (17)	cohen_h_high	95% confidence interval upper bound	float
    
    n.b. Columns 15, 16, and 17 are present when the --regions option is passed, but these columns are on the right side of the table when performing single-site analysis (below). It is generally recommended to use the --header flag and standard CSV parsing to make sure the schema's between experiments are maintained.

When performing single-site analysis, the following additional columns are added:

column	name	description	type
Columns 20 and 21 have the replicate pairwise MAP-based p-values and effect sizes which are calculated based on their order provided on the command line. For example in the abbreviated command below:

In [85]:
experiment_codition="unedit_K562_v_T_day35_noFilter_mC07"

In [78]:
dmr_path="/home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/20251118_single_base_unedit_K562_v_T_day35_noFilter_mC07.bed"
# "/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_35/modkit_dmr/mc_07_noFilter/new_dmr_output/20251109_single_base_noFilter_mC07_day35_CRoff_vs_Unedit.bed"

In [79]:
dmr_path

'/home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/20251118_single_base_unedit_K562_v_T_day35_noFilter_mC07.bed'

In [81]:
out_dir = "/home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/"

In [82]:
# Read DMR BED (robust to header/no-header) and assign canonical column names (uses existing vars: dmr_path, out_dir, date_today, pd, os)
canonical_cols = [
    "chrom", "start", "end", "name", "score", "strand",
    "samplea_counts", "samplea_total", "sampleb_counts", "sampleb_total",
    "samplea_percents", "sampleb_percents",
    "samplea_fraction_modified", "sampleb_fraction_modified",
    "map_pvalue", "effect_size",
    "cohen_h", "cohen_h_low", "cohen_h_high",
]
    # "balanced_map_pvalue", "balanced_effect_size"

# read file with header and fallback to header=None when headers look numeric or columns are unexpected
try:
    dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")

    # dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", engine="python") # , header=0
    # # heuristic: if too many numeric-looking column names, re-read as headerless
    # numeric_headers = sum(1 for c in dmr_df.columns if str(c).strip().isdigit())
    # if numeric_headers >= (len(dmr_df.columns) / 2) or dmr_df.shape[1] < 3:
    #     dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")
except Exception:
    dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")

# assign canonical names up to number of columns present, add generic names for extras
ncols = dmr_df.shape[1]
if ncols <= len(canonical_cols):
    dmr_df.columns = canonical_cols[:ncols]
else:
    extras = [f"col_{i}" for i in range(ncols - len(canonical_cols))]
    dmr_df.columns = canonical_cols + extras

# coerce obvious numeric columns to numeric where present
num_cols_to_try = [
    "start", "end", "score",
    "samplea_total", "sampleb_total",
    "samplea_fraction_modified", "sampleb_fraction_modified",
    "map_pvalue", "effect_size",
    "balanced_map_pvalue", "balanced_effect_size"
]
for c in num_cols_to_try:
    if c in dmr_df.columns:
        dmr_df[c] = pd.to_numeric(dmr_df[c], errors="coerce")

# ensure output directory exists and save parsed table (parquet preferred)
os.makedirs(out_dir, exist_ok=True)
parsed_path = os.path.join(out_dir, f"{date_today}_dmr_parsed.parquet")
try:
    dmr_df.to_parquet(parsed_path, index=False)
    print("Saved parquet:", parsed_path)
except Exception:
    csv_path = os.path.join(out_dir, f"{date_today}_dmr_parsed.csv")
    dmr_df.to_csv(csv_path, index=False)
    print("Parquet not available, saved csv:", csv_path)

print("Loaded DMR:", dmr_path)
print("Assigned columns:", dmr_df.columns.tolist())
print("Shape:", dmr_df.shape)
dmr_df.head()

Parquet not available, saved csv: /home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/20251118_dmr_parsed.csv
Loaded DMR: /home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/20251118_single_base_unedit_K562_v_T_day35_noFilter_mC07.bed
Assigned columns: ['chrom', 'start', 'end', 'name', 'score', 'strand', 'samplea_counts', 'samplea_total', 'sampleb_counts', 'sampleb_total', 'samplea_percents', 'sampleb_percents', 'samplea_fraction_modified', 'sampleb_fraction_modified', 'map_pvalue', 'effect_size', 'cohen_h', 'cohen_h_low', 'cohen_h_high']
Shape: (285, 19)


,chrom,start,end,name,score,strand,samplea_counts,samplea_total,sampleb_counts,sampleb_total,samplea_percents,sampleb_percents,samplea_fraction_modified,sampleb_fraction_modified,map_pvalue,effect_size,cohen_h,cohen_h_low,cohen_h_high
0,chr1,206583090,206583091,.,1.609438,-,m:0,2,m:1,1,m:0.00,m:100.00,0.000000,1.000000,2.615765e-01,-1.000000,-3.141593,0.741137,5.542048
1,chr1,206583173,206583174,.,0.626887,+,m:76,99,m:68,80,m:76.77,m:85.00,0.767677,0.850000,3.696490e-01,-0.082323,-0.210471,-0.084182,0.505125
2,chr1,206583174,206583175,.,0.955966,-,m:74,83,m:86,90,m:89.16,m:95.56,0.891566,0.955556,2.805346e-01,-0.063989,-0.246285,-0.051986,0.544555
3,chr1,206583387,206583388,.,50.950941,+,m:6,105,m:62,85,m:5.71,m:72.94,0.057143,0.729412,0.000000e+00,-0.669412,-1.564701,1.278731,1.850671
4,chr1,206583388,206583389,.,30.686950,-,m:10,52,m:65,75,m:19.23,m:86.67,0.192308,0.866667,2.710000e-14,-0.674359,-1.486086,1.132400,1.839772


In [83]:
import os
from IPython.display import display, HTML

# Visualize all columns from dmr_df and save interactive HTMLs to out_dir
import plotly.express as px
import plotly.graph_objects as go

os.makedirs(out_dir, exist_ok=True)

# Save a table summary
summary = dmr_df.describe(include='all').transpose()
summary_path = os.path.join(out_dir, f"{date_today}_dmr_column_summary.csv")
summary.to_csv(summary_path)

numcols = dmr_df.select_dtypes(include=['number']).columns.tolist()

def _safe_name(name):
    return str(name).replace(os.sep, "_").replace(" ", "_").replace("\t", "_")

# Per-column visualizations
for col in dmr_df.columns:
    safe = _safe_name(col)
    try:
        if col in numcols:
            # Histogram
            fig_h = px.histogram(dmr_df, x=col, nbins=80, title=f"Histogram: {col}")
            # fig_h.write_html(os.path.join(out_dir, f"{date_today}_dmr_hist_{safe}.html"), include_plotlyjs='cdn')
            fig_h.show()

            # Boxplot
            fig_b = px.box(dmr_df, y=col, points="outliers", title=f"Boxplot: {col}")
            # fig_b.write_html(os.path.join(out_dir, f"{date_today}_dmr_box_{safe}.html"), include_plotlyjs='cdn')
            fig_b.show()
        else:
            # Categorical / text: show top value counts (up to 50)
            vc = dmr_df[col].fillna("NA").astype(str).value_counts().head(50)
            if len(vc):
                fig_c = px.bar(x=vc.values[::-1], y=vc.index.astype(str)[::-1], orientation='h',
                               title=f"Top value counts: {col}", labels={'x':'count','y':col})
                fig_c.update_layout(yaxis={'categoryorder':'array','categoryarray':vc.index[::-1].astype(str).tolist()})
                # fig_c.write_html(os.path.join(out_dir, f"{date_today}_dmr_valcounts_{safe}.html"), include_plotlyjs='cdn')
                fig_c.show()
            else:
                # fallback: display empty info
                display(HTML(f"<b>{col}</b>: no values to plot"))
    except Exception as e:
        print(f"Skipped plotting column {col!r} due to error: {e}")

# Correlation heatmap for numeric columns
if len(numcols) >= 2:
    try:
        corr = dmr_df[numcols].corr()
        fig_corr = px.imshow(corr, text_auto=True, aspect="auto", title="Correlation matrix (numeric columns)")
        # fig_corr.write_html(os.path.join(out_dir, f"{date_today}_dmr_correlation_numeric.html"), include_plotlyjs='cdn')
        fig_corr.show()
    except Exception as e:
        print("Failed to create correlation heatmap:", e)

print("Saved summary:", summary_path)
print("Plots saved to:", out_dir)

Saved summary: /home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/20251118_dmr_column_summary.csv
Plots saved to: /home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/


In [86]:
# Select significant CG pairs from DMR results and plot them (new cell at index 69).
# Uses existing notebook variables: dmr_df (parsed modkit dmr), df_roi_stats (pileup ROI stats),
# plotly (px) and out_dir/dmr_folder_path for saving. Does not re-import modules.

# Parameters
pvalue_thresh = 0.05

# ensure numeric columns
dmr_df['map_pvalue'] = pd.to_numeric(dmr_df['map_pvalue'], errors='coerce')
dmr_df['effect_size'] = pd.to_numeric(dmr_df['effect_size'], errors='coerce')
dmr_df['samplea_fraction_modified'] = pd.to_numeric(dmr_df['samplea_fraction_modified'], errors='coerce')
dmr_df['sampleb_fraction_modified'] = pd.to_numeric(dmr_df['sampleb_fraction_modified'], errors='coerce')

# filter significant by MAP-based p-value
sig = dmr_df[dmr_df['map_pvalue'] <= pvalue_thresh].copy()

dmr_df['map_pval_less005'] = dmr_df['map_pvalue'] <= 0.05


# # restrict to ROI positions if df_roi_stats exists
# if 'df_roi_stats' in globals():
#     roi_positions = set(df_roi_stats['start'].astype(int).tolist())
#     sig = sig[sig['start'].isin(roi_positions)].copy()

# quick exit if none
if sig.shape[0] == 0:
    print(f"No significant CG pairs found in ROI at map_pvalue <= {pvalue_thresh}")
else:
    # add convenience cols
    sig['pos'] = sig['start'].astype(str)
    sig['a_perc'] = sig['samplea_fraction_modified'] * 100
    sig['b_perc'] = sig['sampleb_fraction_modified'] * 100
    sig['total_reads'] = sig.get('samplea_total', 0).fillna(0).astype(int) + sig.get('sampleb_total', 0).fillna(0).astype(int)

    # save a table of significant sites
    os.makedirs(out_dir, exist_ok=True)
    sig_table_path = os.path.join(out_dir, f"dmr_significant_p{pvalue_thresh:.3f}_roi.tsv")
    sig.to_csv(sig_table_path, sep='\t', index=False)
    print("Saved significant sites table:", sig_table_path)
    display(sig[['chrom','start','end','strand','map_pvalue','effect_size','a_perc','b_perc','total_reads']].reset_index(drop=True))

    # plot the map_pval_less005 distribution which corresponds to significant sites
    fig_mappval_hist = px.histogram(
        dmr_df,
        x='map_pval_less005',
        nbins=80,
        title=f"MAP-based p-value distribution (highlighting p <= {pvalue_thresh}) <br>{experiment_codition}",
        labels={'map_pval_less005':'MAP-based p-value'}
    )
    fig_mappval_hist.update_layout(height=520)
    mappval_hist_path = os.path.join(out_dir, f"dmr_map_pval_distribution.html")
    # fig_mappval_hist.write_html(mappval_hist_path, include_plotlyjs='cdn')
    fig_mappval_hist.show()
    # print("Saved MAP-based p-value distribution histogram:", mappval_hist_path)

    # plot the percent of significant sites where map_pvalue <= pvalue_thresh is colored red, others blue (color not working)
    # Check https://plotly.com/python/pie-charts/ for coloring instructions
    percent_significant = (sig.shape[0] / dmr_df.shape[0]) * 100
    fig_mappval_pie = px.pie(
        dmr_df,
        names=['Not Significant (p > {})'.format(pvalue_thresh), 'Significant (p <= {})'.format(pvalue_thresh)],
        values=[dmr_df.shape[0] - sig.shape[0],sig.shape[0]],
        title=f"Percentage of significant CGs (map_pvalue <= {pvalue_thresh}): {percent_significant:.2f}% <br>{experiment_codition}",
        # color_discrete_map={'Not Significant (p > {})'.format(pvalue_thresh): 'blue',
        #                     'Significant (p <= {})'.format(pvalue_thresh): 'red'},
        # colors=['blue','red']
    )
    fig_mappval_pie.update_layout(height=520)
    mappval_pie_path = os.path.join(out_dir, f"dmr_map_pval_percentage.html")
    # fig_mappval_pie.write_html(mappval_pie_path, include_plotlyjs='cdn')
    fig_mappval_pie.show()
    # print("Saved MAP-based p-value percentage pie chart:", mappval_pie_path)


    # plot effect size distribution of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh
    fig_effectsize_hist = px.histogram(
        dmr_df,
        x='effect_size',
        nbins=80,
        # add a line break and write experiment_codition into the title
        title=f"Effect size distribution (highlighting significant sites with map_pvalue <= {pvalue_thresh})<br>{experiment_codition}",
        color=(dmr_df['map_pvalue'] <= pvalue_thresh), 
        color_discrete_map={True: 'red', False: 'blue'},
    )
            # labels={'effect_size':'Effect size (A - B)'}

    fig_effectsize_hist.update_layout(height=520)
    effectsize_hist_path = os.path.join(out_dir, f"dmr_effect_size_distribution.html")
    # fig_effectsize_hist.write_html(effectsize_hist_path, include_plotlyjs='cdn')
    fig_effectsize_hist.show()
    # print("Saved effect size distribution histogram:", effectsize_hist_path)        

    # plot effect sizes of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh 
    # add color legend names as 'Significant: map_pvalue <= pvalue_thresh' and 'Not Significant: map_pvalue > pvalue_thresh'  
    fig_effectsize_scatter = px.scatter(
        dmr_df,
        x=dmr_df.index,
        y='effect_size',
        color_discrete_map={True: 'red', False: 'blue'},
        color=(dmr_df['map_pvalue'] <= pvalue_thresh), 
        labels={'effect_size':'Effect size (A - B)','index':'Index',
                'color':f'Significant: map_pvalue <= {pvalue_thresh}'},
        title=f"Effect sizes for all CGs (highlighting significant sites with map_pvalue <= {pvalue_thresh}) <br>{experiment_codition}",
    )
    fig_effectsize_scatter.update_layout(height=520)
    effectsize_scatter_path = os.path.join(out_dir, f"dmr_effect_size_scatter.html")
    # fig_effectsize_scatter.write_html(effectsize_scatter_path, include_plotlyjs='cdn')
    fig_effectsize_scatter.show()
    # print("Saved effect size scatter plot:", effectsize_scatter_path)       

    # bar plot effect sizes of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh 
    fig_effectsize_bar = px.bar(
        dmr_df,
        x=dmr_df.index,         
        y='effect_size',
        color=(dmr_df['map_pvalue'] <= pvalue_thresh),
        labels={'effect_size':'Effect size (A - B)','index':'Index',
                'color':f'Significant: map_pvalue <= {pvalue_thresh}'},
        color_discrete_map={True: 'red', False: 'blue'},
        title=f"Effect sizes for all CGs (n={len(dmr_df)}) (highlighting significant sites with map_pvalue <= {pvalue_thresh}) <br>{experiment_codition}",
    )
    fig_effectsize_bar.update_layout(height=520)
    effectsize_bar_path = os.path.join(out_dir, f"dmr_effect_size_bar.html")
    # fig_effectsize_bar.write_html(effectsize_bar_path, include_plotlyjs='cdn')
    fig_effectsize_bar.show()
    # print("Saved effect size bar plot:", effectsize_bar_path) 


    # Bar: effect size per position (without sorting) with effect size colors 
    # dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
    fig_bar_unsorted = px.bar(
        dmr_df,
        x=dmr_df.index, 
        y='effect_size',        
        color='effect_size',
        title=f"Effect size for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
        labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
    )
            # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
            # x='label',

    fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
    bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
    # fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
    fig_bar_unsorted.show()
    # print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
        
    # Bar: effect size per position (without sorting)
    sig['label'] = sig['pos'] + ":" + sig['strand'].astype(str)
    fig_bar_unsorted = px.bar(
        sig,
        x='label',
        y='effect_size',        
        color='effect_size',
        hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        title=f"Effect size for significant CGs (n={len(sig)}) with map_pvalue <= {pvalue_thresh} <br>{experiment_codition}",
        labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
    )
    fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
    bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
    # fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
    fig_bar_unsorted.show()
    # print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
        

    # Bar: effect size per position (sorted)
    sig_sorted = sig.sort_values('effect_size', ascending=False).copy()
    sig_sorted['label'] = sig_sorted['pos'] + ":" + sig_sorted['strand'].astype(str)
    fig_bar = px.bar(
        sig_sorted,
        x='label',
        y='effect_size',
        color='effect_size',
        hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        title=f"Effect size for significant CGs (n={len(sig_sorted)}) <br>{experiment_codition}",
        labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
    )
    fig_bar.update_layout(xaxis_tickangle=45, height=520)
    bar_path = os.path.join(out_dir, f"dmr_sig_effectsize_p{pvalue_thresh:.3f}.html")
    # fig_bar.write_html(bar_path, include_plotlyjs='cdn')
    fig_bar.show()
    # print("Saved effect-size bar plot:", bar_path)



    # Scatter: sample A vs sample B percent modified (size = total reads, color = effect size)
    fig_scatter = px.scatter(
        sig,
        x='a_perc',
        y='b_perc',
        color='effect_size',
        size='total_reads',
        hover_data=['pos','start','map_pvalue','effect_size','cohen_h'],
        title=f"Significant CGs (map_pvalue <= {pvalue_thresh}) — sample A vs B percent modified <br>{experiment_codition}",
        labels={'a_perc':'Sample A % modified','b_perc':'Sample B % modified'}
    )
    fig_scatter.update_layout(height=520)
    scatter_path = os.path.join(out_dir, f"dmr_sig_scatter_p{pvalue_thresh:.3f}.html")
    # fig_scatter.write_html(scatter_path, include_plotlyjs='cdn')
    fig_scatter.show()
    # print("Saved scatter plot:", scatter_path)


Saved significant sites table: /home/michalula/code/epiCausality/epiCode/modkit_differential_methyl/K562_v_T/mc_07/notFiltered/unedit_K526_v_T_d35/dmr_significant_p0.050_roi.tsv


,chrom,start,end,strand,map_pvalue,effect_size,a_perc,b_perc,total_reads
0,chr1,206583387,206583388,+,0.000000e+00,-0.669412,5.714286,72.941180,190
1,chr1,206583388,206583389,-,2.710000e-14,-0.674359,19.230770,86.666670,127
2,chr1,206583707,206583708,+,0.000000e+00,-0.840000,11.650486,96.153843,207
3,chr1,206583708,206583709,-,0.000000e+00,-0.869167,9.333333,96.250000,155
4,chr1,206583766,206583767,+,0.000000e+00,-0.746984,18.085106,92.783505,191
5,chr1,206583767,206583768,-,4.200000e-15,-0.585897,35.000000,93.589740,158
6,chr1,206584104,206584105,+,7.183910e-06,-0.240000,72.033900,96.000000,218
7,chr1,206584105,206584106,-,1.800579e-02,-0.150104,77.380955,92.391306,176
8,chr1,206584137,206584138,+,8.847685e-09,-0.250000,74.626863,100.000000,243
9,chr1,206584151,206584152,+,6.380128e-03,-0.100000,89.473680,99.009900,234


# TODO: check
- are there really NO diffs between the reads selected with the mC > 70 and mC > 99.5% filtering ??

could be as the mC calles are automatically selected

and in the CRoff the auto threshold 
* in mC > 70 was to 0.79
> Using filter threshold 0.7910156 for C.
* in mC > 99.5 was to  0.79
> Using filter threshold 0.7910156 for C.


and in the Unediter the auto threshold 
* in mC > 70 was to 0.8496
> Using filter threshold 0.8496094 for C.
* in mC > 99.5 was to 0.849
> Using filter threshold 0.8496094 for C.


SAME per condition AUTOMATIC modkit filtering threshold per mC run were set

(NOT 0.995 and not 0.7)

# TODO: check
- are there really NO diffs between the reads selected with the mC > 70 and mC > 99.5% filtering ??

could be as the mC calles are automatically selected

and in the CRoff the auto threshold 
* in mC > 70 was to 0.79
> Using filter threshold 0.7910156 for C.
* in mC > 99.5 was to  0.79
> Using filter threshold 0.7910156 for C.


and in the Unediter the auto threshold 
* in mC > 70 was to 0.8496
> Using filter threshold 0.8496094 for C.
* in mC > 99.5 was to 0.849
> Using filter threshold 0.8496094 for C.


SAME per condition AUTOMATIC modkit filtering threshold per mC run were set

(NOT 0.995 and not 0.7)